In [12]:
import chipwhisperer as cw
import matplotlib.pyplot as plt

n = 4

scope = cw.scope()
target = cw.target(scope)

scope.default_setup()

cw.program_target(scope, cw.programmers.STM32FProgrammer, "./recall-CWNANO.hex")

scope.arm()

# scope.adc.samples = 800
scope.adc.samples = 1000


See https://chipwhisperer.readthedocs.io/en/latest/api.html#firmware-update


Detected known STMF32: STM32F04xxx
Extended erase (0x44), this can take ten seconds or more
Attempting to program 12859 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 12859 bytes


In [13]:
import time
def reset_target(scope): 
    scope.io.nrst = 'low'
    time.sleep(0.01)
    scope.io.nrst = 'high'
    time.sleep(0.01)

def cap_pass_trace(pass_guess):
    ret = ""
#     reset_target(scope)
    num_char = target.in_waiting()
    while num_char > 0:
        ret += target.read(num_char, 16)
        time.sleep(0.01)
        num_char = target.in_waiting()

    scope.arm()
#     target.write(pass_guess)
    target.simpleserial_write('a', pass_guess)

    ret = scope.capture()
    if ret:
        print('Timeout happened during acquisition')

    trace = scope.get_last_trace()
    return trace

In [14]:
import numpy as np

found_elts = []

for pos in range(16):
    ref_msg = bytearray(found_elts)  + bytearray([255]*(16-len(found_elts)))
    ref_trace = cap_pass_trace(ref_msg)
    
    
    temp =[]
    for i in range(0,256):
        msg = bytearray(found_elts) + bytearray([i]) + bytearray([255] * (15-len(found_elts)) )

        trace = cap_pass_trace(msg)
#         diff = int(round(np.sum(np.abs(trace - ref_trace)), -1))
        diff = int(round(np.sum(np.abs(trace - ref_trace))))
        temp.append(diff)
#         print(f"pos: {pos}, i: {i}, diff: {diff}")
        
    temp = np.array(temp)
    mean = temp.mean()
    temp = abs(temp - mean)
    max_elt = temp.argmax()
        
    print(f"position: {pos}, found_elt: {max_elt}")
    found_elts.append(max_elt)


position: 0, found_elt: 207
position: 1, found_elt: 68
position: 2, found_elt: 1
position: 3, found_elt: 69
position: 4, found_elt: 17
position: 5, found_elt: 100
position: 6, found_elt: 37
position: 7, found_elt: 135
position: 8, found_elt: 66
position: 9, found_elt: 57
position: 10, found_elt: 84
position: 11, found_elt: 15
position: 12, found_elt: 143
position: 13, found_elt: 129
position: 14, found_elt: 161
position: 15, found_elt: 218


In [ ]:
# [158, 103, 80, 74, 112, 89, 140, 29, 192, 139, 103, 171, 25, 182, 68, 244]

In [15]:
scope.dis()
target.dis()